In [ ]:
using MuJoCo
using LinearAlgebra
using ForwardDiff
using MatrixEquations
using Revise
using ImpactSafety

In [ ]:
"""
Stabilizing infinite horizon LQR policy
"""

mutable struct InfiniteHorizonLQR
    qref::Vector
    vref::Vector
    uref::Vector
    K::Matrix
end

function InfiniteHorizonLQR(
    qref::Vector,
    vref::Vector,
    uref::Vector,
    Q::Matrix,
    R::Matrix,
    A::Matrix,
    B::Matrix
)::InfiniteHorizonLQR
    nx = size(Q)[1]
    nu = size(R)[1]
    P = zeros(nx, nu)
    _, _, K, _ = ared(A, B, R, Q, P)
    return InfiniteHorizonLQR(qref, vref, uref, K)
end

function (lqr::InfiniteHorizonLQR)(
    model::Model,
    data::Data
)::Nothing
    qerr = zeros(model.nv)
    mj_differentiatePos(model, qerr, 1, lqr.qref, data.qpos)
    verr = data.qvel - lqr.vref
    xerr = [qerr; verr]
    data.ctrl .= lqr.uref .- lqr.K*xerr
    return nothing
end

In [ ]:
"""
Obstacle avoidance auxiliary functions
"""

function opos(o::Vector)::Vector
    return o[1:3]
end

function ovel(o::Vector)::Vector
    return o[4:6]
end

function ptask(xtask::Vector)::Vector
    return xtask[1:3]
end

function vtask(xtask::Vector)::Vector
    return xtask[4:6]
end

function Φ(dmin::Real, n::Real, k::Real, o::Vector, xtask::Vector)::Vector
    rel_dist = opos(o) - ptask(xtask)
    d = norm(rel_dist)
    d_unit = rel_dist / d

    rel_vel = ovel(o) - vtask(xtask)
    ḋ = rel_vel' * d_unit
    return [n*dmin - n*d - k*ḋ]
end

function Φ̇cbf(
    Φargs::Tuple,
    λ::Real,
    model::Model,
    data::Data,
    bodyidx::Int
)::Vector
    # Get task state
    ptask = data.xipos[bodyidx, :]
    vtask = data.cvel[bodyidx, 4:6]
    xtask = [ptask; vtask]
    Φval = Φ(Φargs..., xtask)
    return -λ*Φval
end

function δΦδx(
    Φargs::Tuple,
    model::Model,
    data::Data,
    bodyidx::Int
)::Matrix
    # Get task state
    ptask = data.xipos[bodyidx, :]
    vtask = data.cvel[bodyidx, 4:6]
    xtask = [ptask; vtask]

    # Safety index jacobian w.r.t. task state
    δΦδxtask = ForwardDiff.jacobian(δxtask -> filt.Φ(Φargs..., δxtask), xtask)

    # Task state jacobian w.r.t. joint position
    Jtask = MuJoCo.mj_zeros(3, model.nv)
    mj_jacBodyCom(model, data, Jtask, nothing, bodyidx)

    # Time-derivative of task state jacobian w.r.t. joint position
    Jdottask = taskspace_Jdot(model, data, bodyidx)

    # Task state jacobian w.r.t. joint states
    δxtaskδx = [Jtask zeros(size(Jdottask)); Jdottask Jtask]

    # Safety index jacobian w.r.t. joint states
    return δΦδxtask * δxtaskδx
end

function decoupled_Φ(
    dmin::Real,
    n::Real,
    k::Real,
    o::Vector,
    ptask::Vector,
    vtask::Vector
)::Vector
    rel_dist = opos(o) - ptask
    d = norm(rel_dist)
    d_unit = rel_dist / d

    rel_vel = ovel(o) - vtask
    ḋ = rel_vel' * d_unit
    return [n*dmin - n*d - k*ḋ]
end

In [ ]:
"""
Load model and compute reference control
"""

# Get model and data
model = load_model(joinpath(
    example_model_files_directory(),
    "humanoid",
    "humanoid.xml"
))
data = init_data(model)
reset!(model, data)

# Set sim time step
Δt = 1e-3
model.opt.timestep = Δt

# Propagate derived quantities
mj_forward(model, data)

# Set joint accelerations to 0 and adjust height
data.qacc .= 0
data.qpos[3] += -5.647855e-4

# Get actuator forces from inverse dynamics
mj_inverse(model, data)
println("Desired forces: ", data.qfrc_inverse)

# Get references
uref = pinv(data.actuator_moment)' * data.qfrc_inverse
uref = reshape(uref, length(uref))
qref = copy(data.qpos)
qref = reshape(qref, length(qref))
vref = zeros(model.nv)

In [ ]:
"""
Design cost matrices
"""

nu = model.nu # number of actuators/controls
nv = model.nv # number of degrees of freedom

torso = MuJoCo.body(model, "torso")
left_foot = MuJoCo.body(model, "foot_left")
right_foot = MuJoCo.body(model, "foot_right")

# Get Jacobian for torso CoM
reset!(model, data)
model.opt.timestep = Δt

data.qpos .= qref
forward!(model, data)
jac_com = mj_zeros(3, nv)
mj_jacSubtreeCom(model, data, jac_com, torso.id)

# Get left and right foot Jacobians for balancing
jac_lfoot = mj_zeros(3, nv)
mj_jacBodyCom(model, data, jac_lfoot, nothing, left_foot.id)
jac_rfoot = mj_zeros(3, nv)
mj_jacBodyCom(model, data, jac_rfoot, nothing, right_foot.id)

# Design Q-matrix to balance CoM over foot
jac_diff = jac_com .- jac_lfoot .- jac_rfoot
Qbalance = jac_diff' * jac_diff

# Get indices into relevant sets of joints
free_dofs = 1:6
body_dofs = 7:nv

# Get all the joints using a list comprehension.
# We add one to the raw ID to get the Julia 1-based index of the joint.
abdomen_dofs = [
    jnt.id+1 for jnt in MuJoCo.joints(model) if occursin("abdomen", jnt.name)
]
left_leg_dofs = [
    jnt.id+1 for jnt in MuJoCo.joints(model)
    if occursin("left", jnt.name) && any(
        occursin(part, jnt.name) for part in ("hip", "knee", "ankle")
    )
]
right_leg_dofs = [
    jnt.id+1 for jnt in MuJoCo.joints(model)
    if occursin("right", jnt.name) && any(
        occursin(part, jnt.name) for part in ("hip", "knee", "ankle")
    )
]

balance_dofs = vcat(abdomen_dofs, left_leg_dofs, right_leg_dofs)
other_dofs = setdiff(body_dofs, balance_dofs)

println("Balance dofs: ", balance_dofs)
println("Other dofs:   ", other_dofs)

# Cost coefficients
balance_joint_cost = 1e+1          # Joints can move a bit and still balance
other_joint_cost   = 1e-1        # Other joints can do whatever

# Construct joint Q matrix
Qjoint = Matrix{Float64}(I, nv, nv)
Qjoint[free_dofs, free_dofs] *= 0
Qjoint[balance_dofs, balance_dofs] *= balance_joint_cost
Qjoint[other_dofs, other_dofs] *= other_joint_cost

balance_cost = 2e+2

Qpos = 2e-1*balance_cost*Qbalance + Qjoint
Q = [Qpos zeros(nv,nv); zeros(nv, 2nv)]  + (1e-10) * I # Add ϵI for positive definite Q
R = Matrix{Float64}(I, nu, nu)

In [ ]:
"""
Get dynamics jacobians and LQR gains
"""

# Initialise the model at our set point
reset!(model, data)
model.opt.timestep = Δt
data.ctrl .= uref
data.qpos .= qref

# Finite-difference parameters
ϵ = 1e-6
centered = true

# Compute the Jacobians
A = mj_zeros(2nv, 2nv)
B = mj_zeros(2nv, nu)
mjd_transitionFD(model, data, ϵ, centered, A, B, nothing, nothing)
A = Matrix(A)
B = Matrix(B)

# Infinite horizon LQR policy
lqr = InfiniteHorizonLQR(qref, vref, uref, Q, R, A, B)

In [ ]:
# Safety filter policy
nu = Int(model.nu)
ϵ = 1e-6
filt = mjSafetyFilter(nu, ϵ, Φ, Φ̇cbf, δΦδx)

dmin = 0.4
n = 1
k = 0.5
o = [0, 0, 1, 0, 0, 0]
Φargs = (dmin, n, k, o)

λ = 1e2
Φ̇args = (Φargs, λ)

bodyidx = 3

In [ ]:
"""
Simulate
"""

# Sim horizon
N = 2000

# State history
xhist = zeros(model.nq + model.nv + model.na, N)

# Initial conditions
reset!(model, data)
model.opt.timestep = Δt

zoffset = 2
data.qpos .= qref
data.qpos[3] += zoffset

vic = 0.1*rand(3)
data.qvel[1:3] .= vic
@show vic

forward!(model, data)
for k = 1:N
    xhist[:,k] .= get_physics_state(model, data)
    # Activate controller when robot touches ground
    if data.qpos[3] < qref[3] + 1e-1
        lqr(model, data)
        filt(Φargs, Φ̇args, model, data, bodyidx)
    end
    step!(model, data)
end
@show argmax(data.xipos[:,3])

In [ ]:
init_visualiser()
visualise!(model, data, trajectories=[xhist])